In [1]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-
"""
This process calculates the blockgroupspending csv file
by applying 
  -  the regression models, estimating each target expenditure by blockgroup
  -  the prediction, applying the regression and penetration to ACS

Block Group GEOID,  TOTEXP, FOOD, TRANS ...
_________________   _______ ____  ____

"""
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
import warnings 
warnings.filterwarnings('ignore')

import xgboost as xgb

acs = pd.read_pickle('acs.pkl')
acs = acs[acs.B25003_001E > 0] # 2801 block groups have no households
acs = acs[acs.B19013_001E != -666666666]  # 14,405 rows use this as an NA placeholder
acs = acs[acs.B19013_001E != 250001]      # 2,431 rows use this topcoding

independents = ['income_z',
                'income_A', 'income_B', 'income_C', 'income_D', 'income_E', 'income_F', 'income_G', 
                'homeownership',
                'vehicles_0_1',
                'vehicles_0', 'vehicles_1', 'vehicles_2', 'vehicles_3', 'vehicles_4', 'vehicles_5plus',
                'married',
                'children',
                'bedroomstwofewer',
                'bedrooms_0', 'bedrooms_1', 'bedrooms_2', 'bedrooms_3', 'bedrooms_4', 'bedrooms_5plus',
                'fuel_log',
                'fuel_A', 'fuel_B', 'fuel_C', 'fuel_D', 'fuel_E', 'fuel_F', 'fuel_G',
                'familysize_1','familysize_2','familysize_3','familysize_4','familysize_5','familysize_6','familysize_7more',
                #'income_low','income_lowermid','income_uppermid','income_high',
               ]

# Calculate the independent variables
acs['income_log'] = np.log(acs.B19013_001E)
acs['income_z'] = (acs.income_log-acs.income_log.mean())/acs.income_log.std()
acs['homeownership'] = acs.B25003_002E/acs.B25003_001E  # proportion of occupied housing that is owned 
acs['vehicles_0_1'] = (acs.B25044_010E + acs.B25044_011E + acs.B25044_003E + acs.B25044_004E) / (acs.B25044_002E+acs.B25044_009E) 
acs['vehicles_0'] = acs.B25044_003E  + acs.B25044_010E  
acs['vehicles_1'] = acs.B25044_004E  + acs.B25044_011E   
acs['vehicles_2'] = acs.B25044_005E  + acs.B25044_012E   
acs['vehicles_3'] = acs.B25044_006E  + acs.B25044_013E 
acs['vehicles_4'] = acs.B25044_007E  + acs.B25044_014E  
acs['vehicles_5plus'] = acs.B25044_008E  + acs.B25044_015E   
acs["married"] = acs["B11003_002E"]/acs["B11003_001E"]
acs["children"] = (acs["B11003_003E"]+acs["B11003_010E"]+acs["B11003_016E"])/acs["B11003_001E"]
acs['bedroomstwofewer'] =  (acs.B25041_002E.astype(int)+acs.B25041_003E.astype(int)+acs.B25041_004E.astype(int)) / acs.B25041_001E.astype(int)
acs['bedrooms_0'] = acs.B25041_002E           # No bedroom
acs['bedrooms_1'] = acs.B25041_003E           # 1 bedroom
acs['bedrooms_2'] = acs.B25041_004E           # 2 bedrooms
acs['bedrooms_3'] = acs.B25041_005E           # 3 bedrooms
acs['bedrooms_4'] = acs.B25041_006E           # 4 bedrooms
acs['bedrooms_5plus'] = acs.B25041_007E  # 5 or more bedrooms
acs['fuel_log']  = np.log(1+acs['B25040_001E']-acs['B25040_010E'])
acs['fuel_z']  = (acs['fuel_log']-acs['fuel_log'].mean()) / acs['fuel_log'].std()
acs['familysize_1'] = acs.B11016_010E
acs['familysize_2'] = acs.B11016_003E + acs.B11016_011E
acs['familysize_3'] = acs.B11016_004E + acs.B11016_012E
acs['familysize_4'] = acs.B11016_005E + acs.B11016_013E
acs['familysize_5'] = acs.B11016_006E + acs.B11016_014E
acs['familysize_6'] = acs.B11016_007E + acs.B11016_015E
acs['familysize_7more'] = acs.B11016_008E + acs.B11016_016E
#acs["income_low"]      = np.where(acs.B19013_001E <  50000,1,0)
#acs["income_lowermid"] = np.where((acs.B19013_001E >= 50000)&(acs.B19013_001E <  70000),1,0)
#acs["income_uppermid"] = np.where((acs.B19013_001E >= 70000)&(acs.B19013_001E < 100000),1,0)
#acs["income_high"]     = np.where(acs.B19013_001E >= 100000,1,0)
acs = pd.concat( \
    [acs.reset_index(drop=True),
     pd.get_dummies(\
                     np.where(acs['income_z'] < -1.5, 'A',
                     np.where(acs['income_z'] < -.75, 'B',
                     np.where(acs['income_z'] < -.25, 'C',
                     np.where(acs['income_z'] < .25, 'D',
                     np.where(acs['income_z'] < .75, 'E', 
                     np.where(acs['income_z'] < 1.5, 'F', 'G')))))),prefix="income", dtype=int).reset_index(drop=True),
    pd.get_dummies(\
                     np.where(acs['fuel_z'] < -1.5, 'A',
                     np.where(acs['fuel_z'] < -.75, 'B',
                     np.where(acs['fuel_z'] < -.25, 'C',
                     np.where(acs['fuel_z'] < .25, 'D',
                     np.where(acs['fuel_z'] < .75, 'E', 
                     np.where(acs['fuel_z'] < 1.5, 'F', 'G')))))),prefix="fuel", dtype=int).reset_index(drop=True),
    ], axis=1)


acs['proportion_hispanic'] =  acs.B03003_003E.astype(int) / acs.B03003_001E.astype(int)
acs['proportion_white'] =  (acs.B02001_002E.astype(int) / acs.B02001_001E.astype(int)) - acs['proportion_hispanic']
acs['proportion_black'] =  acs.B02001_003E.astype(int) / acs.B02001_001E.astype(int)
acs['proportion_asian'] =  acs.B02001_005E.astype(int) / acs.B02001_001E.astype(int)
acs['proportion_other'] =  1 - acs.proportion_hispanic - acs.proportion_white - acs.proportion_black - acs.proportion_asian

print("Ready")

Ready


In [2]:
#
# Apply the regression model to 
#

# load model from pickle file
with open("blockgroup_regression_models.pkl", 'rb') as file:  
    models = pickle.load(file)

races = ['white','black','asian','other','hispanic']
targets = ['TOTEXP','FOOD','HOUS','TRANS','HEALTH','RETPEN',]

for target in targets:
    for race in races:
        acs[target+'_'+race] = models[target][race].predict(xgb.DMatrix(acs[independents]))
    # Total expenditure is 
    #      each race's total times each races share
    #      times 4 quarters
    #      times the number of households in the geography
    acs[target] = 1.041 * acs.B11001_001E * 4 *  \
        (acs['proportion_white'] * acs[target+'_white'] + \
         acs['proportion_black'] * acs[target+'_black'] + \
         acs['proportion_asian'] * acs[target+'_asian'] + \
         acs['proportion_other'] * acs[target+'_other'] + \
         acs['proportion_hispanic'] * acs[target+'_hispanic'] )
    print(target,"estimate:","{0:,.2f}".format(acs[target].sum()/(10**12))) 
    # Expecting
    #    TOTEXP 10.4 Trillion
    #    FOOD    1.34
    #    HOUS    3.42
    #    TRANS   1.77
    #    HEALTH  0.83
    #    RETPEN  1.29
    #    OTHER   1.75
acs['OTHER'] = acs['TOTEXP'] - acs[['FOOD','HOUS','TRANS','HEALTH','RETPEN',]].sum(axis=1)
print('OTHER',"estimate:","{0:,.2f}".format(acs['OTHER'].sum()/(10**12))) 

acs[['GEO_ID']+targets+['OTHER']].to_csv('2023blockgroupspending.csv', index=False)

TOTEXP estimate: 10.01
FOOD estimate: 1.51
HOUS estimate: 3.42
TRANS estimate: 2.21
HEALTH estimate: 1.12
RETPEN estimate: 1.15
OTHER estimate: 0.59
